<font size="+0.5">Load libraries and data.</font>

In [ ]:
# Ignore warnings 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
train = pd.read_csv('data/train_accelerometer_features.csv')
val = pd.read_csv('data/val_accelerometer_features.csv')

# <center> Data preprocessing

<font size="+0.5">Label encoding.</font>

<font size="+0.5">Divide data into 3 categories then encode them.</font>

In [ ]:
label_dict = {"Aggressive acceleration": "critical",
             "Aggressive breaking": "critical", 
             "Aggressive left lane change": "significant",
             "Aggressive left turn": "significant",
             "Aggressive right lane change": "significant",
             "Aggressive right turn": "significant",
             "Non-aggressive event": "negligible"}

In [ ]:
def change_label(row):
    return label_dict[row['event']]

In [ ]:
train['event_label'] = train.apply(change_label, axis=1)
val['event_label'] = val.apply(change_label, axis=1)

In [ ]:
encoder = LabelEncoder()
train['event_label_num'] = encoder.fit_transform(train['event_label'])
val['event_label_num'] = encoder.transform(val['event_label'])

In [ ]:
with open("models/encoder.pcl", "wb") as file:
    pickle.dump(encoder, file)

<font size="+0.5">Look at histogramm of the target class.</font>

In [ ]:
train['event_label'].hist();

In [ ]:
val['event_label'].hist();

# <center> Training and evaluating

### <center> Linear models

<font size="+0.5">Split data into train and test</font>

In [ ]:
x_train = train.drop(["event_label_num", "event_label", "event"], axis=1)
y_train = train['event_label_num'].values
x_val = val.drop(["event_label_num", "event_label", "event"], axis=1)
y_val = val['event_label_num'].values

<font size="+0.5">On the histogramm of the target class we see that it isn't balanced. We can't use cutting or dublicating for balancing data, so we dicided to use class weight.</font>

In [ ]:
y = np.append(y_train, y_val)

In [ ]:
class_weight = dict(zip(np.unique(y), compute_class_weight('balanced', np.unique(y), y)))

In [ ]:
clf_lr = LogisticRegression(class_weight=class_weight, random_state = 42)
clf_lr.fit(x_train, y_train)
lr_predict_val = clf_lr.predict(x_val)
lr_predict_train = clf_lr.predict(x_train)
print(f"Train accuracy: {accuracy_score(lr_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(lr_predict_val, y_val)}")

In [ ]:
clf_pac = PassiveAggressiveClassifier(class_weight=class_weight, random_state=42)
clf_pac.fit(x_train, y_train)
pac_predict_val = clf_pac.predict(x_val)
pac_predict_train = clf_pac.predict(x_train)
print(f"Train accuracy: {accuracy_score(pac_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(pac_predict_val, y_val)}")

In [ ]:
clf_svc = SVC(class_weight=class_weight, random_state=42)
clf_svc.fit(x_train, y_train)
svc_predict_val = clf_svc.predict(x_val)
svc_predict_train = clf_svc.predict(x_train)
print(f"Train accuracy: {accuracy_score(svc_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(svc_predict_val, y_val)}")

In [ ]:
clf_pcp = MLPClassifier(random_state=42, solver='lbfgs', max_iter=1000, hidden_layer_sizes=50)
clf_pcp.fit(x_train, y_train)
pcp_predict_val = clf_pcp.predict(x_val)
pcp_predict_train = clf_pcp.predict(x_train)
print(f"Train accuracy: {accuracy_score(pcp_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(pcp_predict_val, y_val)}")

<font size="+0.5">Use GridSearch for find better parameters for best linear models.</font>

In [ ]:
param_svc = {
    "C": np.arange(18, 21, 1),
    'gamma': np.arange(12, 17, 1)
            }
clf_grid = GridSearchCV(SVC(class_weight=class_weight, random_state=42), param_svc, scoring=make_scorer(accuracy_score), verbose=1, n_jobs=-1)
clf_grid.fit(x_train, y_train);

In [ ]:
best_svc_estimator = clf_grid.best_estimator_
grid_predict_val = best_svc_estimator.predict(x_val)
grid_predict_train = best_svc_estimator.predict(x_train)
print(f"Train accuracy: {accuracy_score(grid_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(grid_predict_val, y_val)}")

### <center> Non-linear models

In [ ]:
clf_gbc = GradientBoostingClassifier()
clf_gbc.fit(x_train, y_train)
gbc_predict_val = clf_gbc.predict(x_val)
gbc_predict_train = clf_gbc.predict(x_train)
print(f"Train accuracy: {accuracy_score(gbc_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(gbc_predict_val, y_val)}")

In [ ]:
clf_rfc = RandomForestClassifier(n_estimators=100, class_weight=class_weight, random_state=42)
clf_rfc.fit(x_train, y_train)
rfc_predict_val = clf_rfc.predict(x_val)
rfc_predict_train = clf_rfc.predict(x_train)
print(f"Train accuracy: {accuracy_score(rfc_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(rfc_predict_val, y_val)}")

In [ ]:
clf_dtc = DecisionTreeClassifier(random_state=42, class_weight=class_weight)
clf_dtc.fit(x_train, y_train)
dtc_predict_val = clf_dtc.predict(x_val)
dtc_predict_train = clf_dtc.predict(x_train)
print(f"Train accuracy: {accuracy_score(dtc_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(dtc_predict_val, y_val)}")

<font size="+0.5">Use GridSearch for find better parameters for best non-linear model.</font>

In [ ]:
param_forest = {"n_estimators": np.arange(180, 220, 10),
                "max_depth": np.arange(36, 46, 2)}
clf_grid = GridSearchCV(RandomForestClassifier(n_jobs=-1, class_weight=class_weight, random_state=42), param_forest, scoring=make_scorer(accuracy_score), verbose=1, n_jobs=-1)
clf_grid.fit(x_train, y_train);

In [ ]:
best_forest_estimator = clf_grid.best_estimator_
grid_predict_val = best_forest_estimator.predict(x_val)
grid_predict_train = best_forest_estimator.predict(x_train)
print(f"Train accuracy: {accuracy_score(grid_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(grid_predict_val, y_val)}")

<font size="+0.5">In the end we dicided to check work of the boosting for this task.</font>

In [ ]:
clf_lgb = lgb.LGBMClassifier(
    n_estimators = 800,
    seed = 42,
    learning_rate = 0.015,
    metric = 'multi_logloss',
    objective = 'multiclass',
    early_stopping = 20)

In [ ]:
clf_lgb.fit(X=x_train, y=y_train, eval_set=(x_val, y_val), verbose=0)
lgb_predict_val = clf_lgb.predict(x_val)
lgb_predict_train = clf_lgb.predict(x_train)
print(f"Train accuracy: {accuracy_score(lgb_predict_train, y_train)}")
print(f"Validation accuracy: {accuracy_score(lgb_predict_val, y_val)}")

<font size="+0.5">Save best models.</font>

In [ ]:
with open("models/non-linear-accelerometer-features.pcl", "wb") as file:
    pickle.dump(clf_rfc, file)

In [ ]:
with open("models/linear-accelerometer-features.pcl", "wb") as file:
    pickle.dump(clf_pcp, file)